In [1]:
!pip install pqdm

  Using cached pqdm-0.2.0-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached bounded_pool_executor-0.0.3-py3-none-any.whl.metadata (2.7 kB)
Using cached pqdm-0.2.0-py2.py3-none-any.whl (6.8 kB)
Using cached bounded_pool_executor-0.0.3-py3-none-any.whl (3.4 kB)


In [1]:
import openai
from openai import OpenA
from datasets import load_dataset
import random
import os
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd

# 병렬처리를 위한 pqdm
# pip install pqdm
from pqdm.processes import pqdm

# -------------------------
# (1) 환경설정 및 함수 정의
# -------------------------
load_dotenv()
model_name = "gpt-4o-mini"
api_key = os.environ.get("OPENAI_API_KEY")

# 시드 고정
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# (2) 데이터셋 로드
ds = load_dataset("daekeun-ml/naver-news-summarization-ko", split="train")
sample_column = "summary"

# (2-1) 예시 프롬프트들

I
client = OpenAI(
  api_key=api_key
)

corrot_prompt = """당신은 가상의 중고나라 게시물을 생성해야 합니다.

이름과 연락처(전화번호, 카카오톡, 거래 위치 등...)를 남기고 어떤 상품을 판매하고 있는지를 작성하세요.

- 전화번호는 반드시 5자리(010-87590-12345) 형식으로 작성하고, **로 감싸세요. (예시: *010-87590-12345*)
- 이메일은 실제 존재하지 않도록 `_sudo_`를 포함하여 작성하세요. (예시: *en_sudo_user@example.com*, *user_sudo@semi.com* 등 다양한 패턴)
- 카카오톡, 트위터, 텔레그램 등 다양한 아이디를 사용하되 사용되는 아이디에는 반드시 실제 존재하지 않도록 `_sudo_`를 포함하세요. (예시: *data_insight_sudo* 등 다양한 패턴)
- 주소를 생성할 때는 반드시 **로 감싸세요. (예시: *서울시 강남구 테헤란로 123, ABC 강의실*)
- 글머리 기호를 써도 되지만 남용해선 안 됩니다. 일반적인 블로그, 카페 말투를 쓰십시오.
- 개인정보나 연락처를 작성할 때만큼은 글머리 기호나 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오.

당신이 참고할 게시물입니다. 아래 게시물로부터 아이디어를 얻고 작성하세요.
---
{new_dataset}
---

작성:"""

event_prompt = """당신은 가상의 이벤트 홍보글을 생성해야 합니다.

이벤트명, 일시, 장소, 참가 방법, 주최자 연락처(이메일, 전화번호 등), 이벤트 상세 내용 등을 작성하세요.

- 전화번호는 반드시 5자리(010-87590-12345) 형식으로 작성하고, **로 감싸세요. (예시: *010-87590-12345*)
- 이메일은 실제 존재하지 않도록 `_sudo_`를 포함하여 작성하세요. (예시: *en_sudo_user@example.com*, *user_sudo@semi.com* 등 다양한 패턴)
- 카카오톡, 트위터, 텔레그램 등 다양한 아이디를 사용하되 사용되는 아이디에는 반드시 실제 존재하지 않도록 `_sudo_`를 포함하세요. (예시: *data_insight_sudo* 등 다양한 패턴)
- 주소를 생성할 때는 반드시 **로 감싸세요. (예시: *서울시 강남구 테헤란로 123, ABC 강의실*)
- 글머리 기호를 써도 되지만 남용해선 안 됩니다. 친근하고 설득력 있는 말투를 사용하십시오.
- 연락처나 개인정보를 작성할 때는 글머리 기호나 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오.

당신이 참고할 이벤트 홍보글입니다. 아래 게시물로부터 아이디어를 얻고 작성하세요.
---
{new_dataset}
---

작성:"""

lecture_prompt = """당신은 가상의 강의 또는 세미나 홍보글을 생성해야 합니다.

강의명, 강사명, 일시, 장소, 수강료, 신청 방법, 주최자 연락처(이메일, 전화번호 등), 강의 상세 내용 등을 작성하세요.

- 전화번호는 반드시 5자리(010-87590-12345) 형식으로 작성하고, **로 감싸세요. (예시: *010-87590-12345*)
- 이메일은 실제 존재하지 않도록 `_sudo_`를 포함하여 작성하세요. (예시: *en_sudo_user@example.com*, *user_sudo@semi.com* 등 다양한 패턴)
- 카카오톡, 트위터, 텔레그램 등 다양한 아이디를 사용하되 사용되는 아이디에는 반드시 실제 존재하지 않도록 `_sudo_`를 포함하세요. (예시: *data_insight_sudo* 등 다양한 패턴)
- 주소를 생성할 때는 반드시 **로 감싸세요. (예시: *서울시 강남구 테헤란로 123, ABC 강의실*)
- 글머리 기호를 써도 되지만 남용해선 안 됩니다. 전문적이고 신뢰감 있는 말투를 사용하십시오.
- 연락처나 개인정보를 작성할 때는 글머리 기호나 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오.

당신이 참고할 강의/세미나 홍보글입니다. 아래 게시물로부터 아이디어를 얻고 작성하세요.
---
{new_dataset}
---

작성:"""

product_prompt = """당신은 가상의 제품 리뷰 또는 홍보글을 생성해야 합니다.

제품명, 가격, 구매처, 제품 특징, 사용 후기, 연락처(구매 문의처 등) 등을 작성하세요.

- 전화번호는 반드시 5자리(010-87590-12345) 형식으로 작성하고, **로 감싸세요. (예시: *010-87590-12345*)
- 이메일은 실제 존재하지 않도록 `_sudo_`를 포함하여 작성하세요. (예시: *en_sudo_user@example.com*, *user_sudo@semi.com* 등 다양한 패턴)
- 카카오톡, 트위터, 텔레그램 등 다양한 아이디를 사용하되 사용되는 아이디에는 반드시 실제 존재하지 않도록 `_sudo_`를 포함하세요. (예시: *data_insight_sudo* 등 다양한 패턴)
- 주소를 생성할 때는 반드시 **로 감싸세요. (예시: *서울시 강남구 테헤란로 123, ABC 강의실*)
- 글머리 기호를 써도 되지만 남용해선 안 됩니다. 솔직하고 친근한 말투를 사용하십시오.
- 연락처나 개인정보를 작성할 때는 글머리 기호나 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오.

당신이 참고할 제품 리뷰/홍보글입니다. 아래 게시물로부터 아이디어를 얻고 작성하세요.
---
{new_dataset}
---

작성:"""

meeting_prompt = """당신은 가상의 모임 또는 동호회 홍보글을 생성해야 합니다.

모임명, 활동 내용, 모임 일시, 장소, 가입 방법, 연락처(이메일, 전화번호 등) 등을 작성하세요.

- 전화번호는 반드시 5자리(010-87590-12345) 형식으로 작성하고, **로 감싸세요. (예시: *010-87590-12345*)
- 이메일은 실제 존재하지 않도록 `_sudo_`를 포함하여 작성하세요. (예시: *en_sudo_user@example.com*, *user_sudo@semi.com* 등 다양한 패턴)
- 카카오톡, 트위터, 텔레그램 등 다양한 아이디를 사용하되 사용되는 아이디에는 반드시 실제 존재하지 않도록 `_sudo_`를 포함하세요. (예시: *data_insight_sudo* 등 다양한 패턴)
- 주소를 생성할 때는 반드시 **로 감싸세요. (예시: *서울시 강남구 테헤란로 123, ABC 강의실*)
- 글머리 기호를 써도 되지만 남용해선 안 됩니다. 친근하고 열정적인 말투를 사용하십시오.
- 연락처나 개인정보를 작성할 때는 글머리 기호나 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오.

당신이 참고할 모임/동호회 홍보글입니다. 아래 게시물로부터 아이디어를 얻고 작성하세요.
---
{new_dataset}
---

작성:"""

anonymization_prompt = """다음 텍스트의 개인정보를 아래 규칙에 따라 비식별화해주세요:

1. 이메일 주소는 @앞부분을 X로 마스킹하고 도메인은 유지해주세요.
   예시: example@gmail.com -> *[EMAIL]*

2. 전화번호는 모두 숫자 부분을 X로 변경해주세요.
   예시: 010-12334-56783 -> *[PHONE_NUMBER]*

3. 카카오톡 ID, 사용자 ID 등의 ID 전체를 X로 마스킹해주세요.
   예시: user_sudo123 -> *[ID]*
   예시: @SmartSec_sudo -> *[ID]*

4. 주소는 아래 예시와 같이 제거해주세요.
   예시: 거래는 *서울 강남역* -> *[ADDR]*
   예시: *서울시 강남구 청담동 123-45, 청담 커뮤니티 센터 2층* -> *[ADDR]*
   예시: *서울시 강남구 테헤란로 123, ABC 강의실* -> *[ADDR]*
   예시: *서울시 청년센터 1층 대강당* -> *[ADDR]*

5. 사람 이름도 마스킹해주세요. 
   예시: 홍길동 -> *[NAME]*

6. 비식별화 예시 
### 원본 데이터 
# **이벤트명: 우리 동네 청년 창업 페스티벌**

안녕하세요! 여러분을 위한 특별한 기회를 소개합니다. 창업에 관심 있는 청년들을 위한 '우리 동네 청년 창업 페스티벌'이 열립니다! 

**일시:** 2023년 11월 15일 (수) 오전 10시부터 오후 5시까지  
**장소:** *서울시 청년센터 1층 대강당*

이번 페스티벌에서는 창업 전문가의 강연, 실전 창업 멘토링, 네트워킹 기회를 제공합니다. 다양한 부스와 체험 프로그램을 통해 창업에 대한 실질적인 정보를 얻고, 새로운 아이디어를 얻어가실 수 있습니다. 참여하시는 모든 분들께 소정의 기념품도 제공하니 많은 관심 부탁드립니다!

참가 방법은 간단합니다. 아래의 이메일로 참여 신청을 해주시면 됩니다. 신청은 선착순으로 마감되니 서둘러주세요. 이메일은 *youth_festival_sudo@example.com*입니다. 

혹시 궁금한 점이 있으신가요? 주최자에게 연락 주시면 친절히 안내해 드리겠습니다. 전화번호는 *02-12345-67890*이며, 카카오톡 아이디는 *youth_event_sudo*입니다. 텔레그램으로는 *youth_festival_sudo*를 통해 소통하실 수 있습니다. 트위터에서는 *youth_festival_sudo*를 검색해 주시면 됩니다.

여러분의 참여를 기다립니다! 창업의 꿈을 실현하는 첫걸음을 함께해요!

### 비식별화 데이터 
**이벤트명: 우리 동네 청년 창업 페스티벌**

안녕하세요! 여러분을 위한 특별한 기회를 소개합니다. 창업에 관심 있는 청년들을 위한 '우리 동네 청년 창업 페스티벌'이 열립니다! 

**일시:** 2023년 11월 15일 (수) 오전 10시부터 오후 5시까지  
**장소:** *[ADDR]*

이번 페스티벌에서는 창업 전문가의 강연, 실전 창업 멘토링, 네트워킹 기회를 제공합니다. 다양한 부스와 체험 프로그램을 통해 창업에 대한 실질적인 정보를 얻고, 새로운 아이디어를 얻어가실 수 있습니다. 참여하시는 모든 분들께 소정의 기념품도 제공하니 많은 관심 부탁드립니다!

참가 방법은 간단합니다. 아래의 이메일로 참여 신청을 해주시면 됩니다. 신청은 선착순으로 마감되니 서둘러주세요. 이메일은 *[EMAIL]*입니다. 

혹시 궁금한 점이 있으신가요? 주최자에게 연락 주시면 친절히 안내해 드리겠습니다. 전화번호는 *[PHONE_NUMBER]*이며, 카카오톡 아이디는 *[ID]*입니다. 텔레그램으로는 *[ID]*를 통해 소통하실 수 있습니다. 트위터에서는 *[ID]*를 검색해 주시면 됩니다.

여러분의 참여를 기다립니다! 창업의 꿈을 실현하는 첫걸음을 함께해요!
---
시작:
{new_dataset}

위 규칙에 따라 개인정보가 비식별화된 텍스트를 생성해주세요."""

prompts = [
    corrot_prompt,
    event_prompt,
    lecture_prompt,
    product_prompt,
    meeting_prompt
]

def format_prompt(base_prompt: str, reference_text: str) -> str:
    return base_prompt.format(new_dataset=reference_text)

def generate_text_with_prompt(prompt_text: str) -> str:
    """
    ChatCompletion API 호출 및 응답 텍스트 반환.
    """
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {
                    "role": "system", 
                    "content": "You are ChatGPT, a large language model trained by OpenAI."
                },
                {
                    "role": "user", 
                    "content": prompt_text
                }
            ],
            temperature=0.7,
            max_tokens=512
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"OpenAI API Error: {e}")
        return ""

def anonymize_text(original_text: str) -> str:
    """
    비식별화용 프롬프트 생성 후 ChatCompletion API 호출 및 결과 반환
    """
    anonym_prompt = format_prompt(anonymization_prompt, original_text)
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {
                    "role": "system", 
                    "content": "당신은 비식별화 전문가입니다. 주어진 원문을 규칙에 맞춰 비식별화하세요."
                },
                {
                    "role": "user", 
                    "content": anonym_prompt
                }
            ],
            temperature=0.7,
            max_tokens=512
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"OpenAI API Error (anonymization): {e}")
        return ""

# -------------------------
# (3) 병렬처리할 작업 함수
# -------------------------
def generate_and_anonymize(_):
    """
    1건에 대해 무작위 참조 데이터 추출 -> 프롬프트 선택 -> 생성 -> 비식별화 과정을 진행하고 결과를 반환
    _ : range(sample_size)에서 넘어오는 인덱스(실제로 사용하지 않아도 됨)
    """
    # 무작위 샘플 1개
    random_idx = random.randint(0, len(ds) - 1)
    reference_item = ds[random_idx][sample_column]

    # 5가지 프롬프트 중 하나를 랜덤 선택
    selected_prompt = random.choice(prompts)

    # 프롬프트 생성 및 텍스트 생성
    prompt_for_openai = format_prompt(selected_prompt, reference_item)
    generated_text = generate_text_with_prompt(prompt_for_openai)
    
    # 생성된 텍스트 비식별화
    anonymized_text = anonymize_text(generated_text)

    # 반환 형식은 원하는 구조대로
    return {
        "reference_text": reference_item,
        "generated_text": generated_text,
        "비식별화된 데이터": anonymized_text
    }

# -------------------------
# (4) 실제 병렬 실행
# -------------------------
if __name__ == "__main__":
    sample_size = 4  # 생성할 개수

    # 병렬 실행. n_jobs는 원하는 프로세스(또는 쓰레드) 개수 조절
    # 'argument_type="args"'는 generate_and_anonymize 함수에 단순히 인자 하나씩 넘겨주는 모드
    generated_data = pqdm(
        [i for i in range(sample_size)],
        generate_and_anonymize,
        n_jobs=4
    )

    # -------------------------
    # (5) 결과 확인 및 저장
    # -------------------------
    # 앞부분 5~6개만 확인
    for idx, item in enumerate(generated_data[:2]):
        print(f"=== 샘플 {idx+1} ===")
        print("[원문(참조):]")
        print(item["reference_text"])
        print("\n[생성된 텍스트:]")
        print(item["generated_text"])
        print("\n[비식별화된 데이터:]")
        print(item["비식별화된 데이터"])
        print("-" * 40)

    print("데이터를 모두 생성하였습니다.")


/root/anaconda3/envs/chapter10/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
QUEUEING TASKS | : 100%|██████████| 4/4 [00:00<00:00, 16.97it/s]
PROCESSING TASKS | : 100%|██████████| 4/4 [00:15<00:00,  3.86s/it]
COLLECTING RESULTS | : 100%|██████████| 4/4 [00:00<00:00, 12418.37it/s]

=== 샘플 1 ===
[원문(참조):]
신용카드 플랫폼 카드고릴라가 4일 발표한 '2022년 상반기 인기 체크카드 TOP 10'에 따르면 무조건 결제금액의 0.2%가 적립되고 매월 10일은 응카데이로 1만원 한도 내에서 10% 적립 받을 수 있는 ‘우리 010페이 PAY 체크카드’가 1위를 차지했다.

[생성된 텍스트:]
**강의명:** 효과적인 데이터 분석을 위한 SQL 마스터 클래스  
**강사명:** 김재훈 박사  
**일시:** 2023년 11월 15일(수) 오후 2시 ~ 5시  
**장소:** *서울시 강남구 테헤란로 456, 데이터 분석 센터 3층 세미나실*  
**수강료:** 30,000원  

**신청 방법:** 아래의 이메일로 신청해 주시기 바랍니다. 선착순으로 진행되며, 정원이 마감되면 추가 신청은 받지 않습니다. 신청 후 수강료를 입금하신 분에 한하여 최종 등록이 완료됩니다.  
**주최자 연락처:** *data_analysis_sudo@example.com* / *010-87590-12345*

**강의 상세 내용:**  
이 강의는 데이터 분석에 있어 필수적인 SQL 기초부터 고급 기술까지 아우르는 프로그램입니다. 데이터베이스 설계, 쿼리 작성, 데이터 추출 및 분석 방법론 등을 실습 중심으로 배울 수 있습니다. 강의는 이론과 실습을 병행하여 진행되며, 실제 사례를 통해 이해도를 높이고 현업에서의 활용 방안을 모색합니다. SQL의 기본 문법, JOIN, GROUP BY, 서브쿼리, 데이터 집계 방법 등을 배우며, 강의 후에는 데이터 분석 능력을 한층 더 끌어올릴 수 있을 것입니다.

추가 정보나 문의 사항이 있으시면 카카오톡 *data_analysis_sudo* 또는 텔레그램 *data_expert_sudo*로 연락 주시기 바랍니다. 많은 관심과 참여 부탁드립니다.

[비식별화된 데이터:]
### 비식별화 데이터 
**강의명:** 효과적인 데이터 분석을 위한 SQL 마스터 클래스  
**강사명:** *[NAME]*  
**일시:** 2